In [ ]:
from IPython.display import HTML, display, Javascript
import os
import glob
import base64
import json

def create_media_gallery():
    # Find all media files
    mp4_files = glob.glob('./ComfyUI/output/*.mp4', recursive=True)
    webp_files = glob.glob('./ComfyUI/output/*.webp', recursive=True)
    
    # Sort all files
    all_files = sorted(mp4_files + webp_files)
    
    # Get corresponding thumbnail images
    thumbnails = {}
    for file_path in all_files:
        base_name = os.path.splitext(file_path)[0]
        png_file = f"{base_name}.png"
        
        if os.path.exists(png_file):
            thumbnails[file_path] = png_file
    
    if not all_files:
        return HTML("<p>No media files found</p>")
    
    # Create unique IDs for this gallery instance
    import random
    gallery_id = f"gallery_{random.randint(10000, 99999)}"
    preview_id = f"preview_{random.randint(10000, 99999)}"
    container_id = f"container_{random.randint(10000, 99999)}"
    selected_id = f"selected_{random.randint(10000, 99999)}"
    
    # Create a list of files for JS
    files_json = json.dumps([{"path": f, "name": os.path.basename(f)} for f in all_files])
    
    # Define handler for file deletion
    def delete_handler(indices):
        results = []
        for idx in indices:
            if 0 <= idx < len(all_files):
                file_path = all_files[idx]
                try:
                    if os.path.exists(file_path):
                        # Delete the main file
                        os.remove(file_path)
                        
                        # Also try to delete thumbnail if it exists
                        base_name = os.path.splitext(file_path)[0]
                        png_file = f"{base_name}.png"
                        if os.path.exists(png_file):
                            try:
                                os.remove(png_file)
                            except:
                                pass  # Ignore errors if cannot delete thumbnail
                        results.append({"idx": idx, "path": file_path, "success": True})
                    else:
                        results.append({"idx": idx, "path": file_path, "success": False, "error": "File not found"})
                except Exception as e:
                    results.append({"idx": idx, "path": file_path, "success": False, "error": str(e)})
            else:
                results.append({"idx": idx, "success": False, "error": "Invalid index"})
        return results
    
    # Build the HTML with two-column layout
    html_code = f"""
    <style>
    #{container_id} {{
        display: flex;
        width: 100%;
        gap: 20px;
        height: 600px;
        margin-bottom: 20px;
        max-width: 100%;
        box-sizing: border-box;
    }}
    
    #{gallery_id} {{
        flex: 0 0 60%;
        display: flex;
        flex-wrap: wrap;
        gap: 15px;
        overflow-y: auto;
        overflow-x: hidden;
        padding: 10px;
        border: 1px solid #ddd;
        border-radius: 4px;
        background-color: #f8f9fa;
        align-content: flex-start;
        height: 100%;
        box-sizing: border-box;
    }}
    
    #{preview_id} {{
        flex: 0 0 38%;
        padding: 10px;
        border: 1px solid #ddd;
        border-radius: 4px;
        background-color: #f8f9fa;
        position: sticky;
        top: 20px;
        height: 100%;
        overflow-y: auto;
        overflow-x: hidden;
        display: flex;
        flex-direction: column;
        justify-content: flex-start;
        align-items: center;
        box-sizing: border-box;
    }}
    
    .thumbnail-container {{
        position: relative;
        margin-bottom: 10px;
        width: 180px;
        box-sizing: border-box;
    }}
    
    .thumbnail {{
        width: 180px;
        height: 135px;
        background-color: #f0f0f0;
        border: 1px solid #ddd;
        display: flex;
        justify-content: center;
        align-items: center;
        cursor: pointer;
        position: relative;
        overflow: hidden;
        border-radius: 4px;
        box-sizing: border-box;
    }}
    
    .thumbnail.selected {{
        border: 3px solid #007bff;
    }}
    
    .thumbnail.marked-delete {{
        border: 3px solid #dc3545;
    }}
    
    .thumbnail:hover {{
        border-color: #007bff;
        box-shadow: 0 0 5px rgba(0,123,255,0.5);
    }}
    
    .thumbnail img {{
        width: 100%;
        height: 100%;
        object-fit: cover;
    }}
    
    .icon-placeholder {{
        font-size: 48px;
        display: flex;
        justify-content: center;
        align-items: center;
        height: 100%;
        width: 100%;
        background-color: #e9ecef;
    }}
    
    .thumbnail span {{
        position: absolute;
        bottom: 0;
        left: 0;
        right: 0;
        background: rgba(0,0,0,0.7);
        color: white;
        padding: 4px 8px;
        font-size: 12px;
        white-space: nowrap;
        overflow: hidden;
        text-overflow: ellipsis;
    }}
    
    .thumbnail-number {{
        position: absolute;
        top: 5px;
        left: 5px;
        background: rgba(0,0,0,0.7);
        color: white;
        padding: 2px 6px;
        border-radius: 50%;
        font-size: 14px;
        font-weight: bold;
        z-index: 10;
    }}
    
    .thumbnail-actions {{
        position: absolute;
        top: 5px;
        right: 5px;
        display: none;
        background: rgba(0,0,0,0.6);
        border-radius: 3px;
        padding: 2px;
    }}
    
    .thumbnail-container:hover .thumbnail-actions {{
        display: flex;
    }}
    
    .action-btn {{
        background: none;
        border: none;
        color: white;
        cursor: pointer;
        margin: 0 2px;
        padding: 2px 5px;
        font-size: 16px;
    }}
    
    .action-btn:hover {{
        color: #ff6b6b;
    }}
    
    .controls {{
        display: flex;
        gap: 10px;
        margin-bottom: 15px;
        flex-wrap: wrap;
    }}
    
    .btn {{
        padding: 6px 12px;
        background-color: #007bff;
        color: white;
        border: none;
        border-radius: 4px;
        cursor: pointer;
    }}
    
    .btn:hover {{
        background-color: #0069d9;
    }}
    
    .btn-danger {{
        background-color: #dc3545;
    }}
    
    .btn-danger:hover {{
        background-color: #c82333;
    }}
    
    .status-msg {{
        margin-top: 10px;
        padding: 8px 12px;
        border-radius: 4px;
        display: none;
    }}
    
    .status-success {{
        background-color: #d4edda;
        color: #155724;
    }}
    
    .status-error {{
        background-color: #f8d7da;
        color: #721c24;
    }}
    
    .preview-content {{
        max-width: 100%;
        text-align: center;
    }}
    
    .preview-content img {{
        max-width: 100%;
        max-height: 350px;
        object-fit: contain;
    }}
    
    .preview-content video {{
        max-width: 100%;
        max-height: 350px;
    }}
    
    .preview-placeholder {{
        color: #6c757d;
        text-align: center;
        font-style: italic;
    }}
    
    .preview-title {{
        margin-top: 10px;
        font-weight: bold;
        text-align: center;
        word-break: break-word;
        max-width: 100%;
    }}

    .selected-container {{
        width: 100%;
        margin-top: 20px;
        padding: 10px;
        background-color: #f0f0f0;
        border-radius: 4px;
    }}

    .selected-title {{
        font-weight: bold;
        margin-bottom: 10px;
    }}

    .selected-code {{
        background-color: #f8f9fa;
        border: 1px solid #ddd;
        border-radius: 4px;
        padding: 10px;
        font-family: monospace;
        overflow-x: auto;
        white-space: pre;
        margin-bottom: 10px;
    }}

    .clear-btn {{
        padding: 4px 8px;
        background-color: #6c757d;
        color: white;
        border: none;
        border-radius: 4px;
        cursor: pointer;
        font-size: 12px;
    }}

    .clear-btn:hover {{
        background-color: #5a6268;
    }}
    </style>
    
    <div class="controls">
        <button id="clear-selection-btn" class="btn" onclick="window.galleryApp.clearSelection()" disabled>Clear Selection</button>
        <button id="download-selected-btn" class="btn" onclick="window.galleryApp.downloadSelected()" disabled>Download Selected</button>
    </div>
    
    <div id="status-message" class="status-msg"></div>
    
    <div id="{container_id}">
        <div id="{gallery_id}">
    """
    
    # Add thumbnails
    for idx, file_path in enumerate(all_files):
        file_name = os.path.basename(file_path)
        ext = os.path.splitext(file_path)[1].lower()
        thumbnail_img = thumbnails.get(file_path, "")
        
        # Create thumbnail content
        if thumbnail_img:
            thumbnail_html = f"""<img src="{thumbnail_img}" alt="{file_name}">"""
        else:
            # Use icon if no thumbnail image found
            if ext == '.mp4':
                icon = "🎬"  # Video icon
            elif ext == '.webp':
                icon = "🖼️"  # Image icon
            else:
                icon = "📄"  # Generic file icon
                
            thumbnail_html = f"""<div class="icon-placeholder">{icon}</div>"""
            
        html_code += f"""
        <div class="thumbnail-container" data-idx="{idx}" data-path="{file_path}">
            <div class="thumbnail-number">{idx + 1}</div>
            <div class="thumbnail" onclick="window.galleryApp.previewFile({idx})">
                {thumbnail_html}
                <span>{file_name}</span>
            </div>
            <div class="thumbnail-actions">
                <button class="action-btn" onclick="window.galleryApp.toggleDeleteMark({idx}, event)">🗑️</button>
                <button class="action-btn" onclick="window.galleryApp.downloadFile('{file_path}', event)">⬇️</button>
            </div>
        </div>
        """
    
    # Close the gallery div and add the preview div
    html_code += f"""
        </div>
        <div id="{preview_id}">
            <div class="preview-placeholder">
                <p>Click on a thumbnail to preview</p>
            </div>
            <div id="{selected_id}" class="selected-container" style="display: none;">
                <div class="selected-title">Selected for deletion:</div>
                <div class="selected-code"></div>
                <button class="clear-btn" onclick="window.galleryApp.clearSelection()">Clear Selection</button>
            </div>
        </div>
    </div>
    """
    
    # Add hidden preview content divs
    for idx, file_path in enumerate(all_files):
        ext = os.path.splitext(file_path)[1].lower()
        file_name = os.path.basename(file_path)
        
        if ext == '.mp4':
            preview_content = f"""
            <div class="preview-content">
                <video controls autoplay>
                    <source src="{file_path}" type="video/mp4">
                    Your browser does not support the video tag.
                </video>
                <div class="preview-title">{file_name}</div>
            </div>
            """
        elif ext == '.webp':
            preview_content = f"""
            <div class="preview-content">
                <img src="{file_path}">
                <div class="preview-title">{file_name}</div>
            </div>
            """
        else:
            preview_content = f"""
            <div class="preview-content">
                <p>Unsupported file type: {ext}</p>
                <div class="preview-title">{file_name}</div>
            </div>
            """
            
        html_code += f"""
        <div id="preview_content_{idx}" style="display:none;">
            {preview_content}
        </div>
        """
    
    # Add JavaScript with proper preview handling - using an actual template string literal in JS
    js_code = """
    <script>
    // Create a namespace for our gallery app to avoid variable scope issues
    window.galleryApp = (function() {
        // Store all files
        const allFiles = ALLFILES_JSON_PLACEHOLDER;
        let currentSelectedIdx = -1;
        let selectedForDeletion = [];
        
        // Function to preview a file
        function previewFile(idx) {
            // Update selected thumbnail styling
            if (currentSelectedIdx >= 0) {
                const prevSelected = document.querySelector(`[data-idx="${currentSelectedIdx}"] .thumbnail`);
                if (prevSelected) {
                    prevSelected.classList.remove('selected');
                    // Maintain marked-delete class if it's selected for deletion
                    if (selectedForDeletion.includes(currentSelectedIdx)) {
                        prevSelected.classList.add('marked-delete');
                    }
                }
            }
            
            const newSelected = document.querySelector(`[data-idx="${idx}"] .thumbnail`);
            if (newSelected) {
                newSelected.classList.add('selected');
                // Keep the marked-delete class if it's in our deletion array
                if (selectedForDeletion.includes(idx)) {
                    newSelected.classList.add('marked-delete');
                }
            }
            currentSelectedIdx = idx;
            
            // Use hidden preview content divs
            const previewDiv = document.getElementById('PREVIEW_ID_PLACEHOLDER');
            const previewContent = document.getElementById(`preview_content_${idx}`).innerHTML;
            
            // Replace only the main preview part, keeping the selected-for-deletion part
            const selectedContainer = document.getElementById('SELECTED_ID_PLACEHOLDER');
            previewDiv.innerHTML = previewContent;
            previewDiv.appendChild(selectedContainer);
            
            // Make sure selected container is visible if we have items
            updateSelectedDisplay();
        }
        
        // Function to toggle delete mark
        function toggleDeleteMark(idx, event) {
            if (event) event.stopPropagation();
            
            const thumbnail = document.querySelector(`[data-idx="${idx}"] .thumbnail`);
            const index = selectedForDeletion.indexOf(idx);
            
            if (index === -1) {
                // Add to delete selection
                selectedForDeletion.push(idx);
                thumbnail.classList.add('marked-delete');
            } else {
                // Remove from delete selection
                selectedForDeletion.splice(index, 1);
                
                // Keep the selection class if it's the current preview
                if (idx === currentSelectedIdx) {
                    thumbnail.classList.remove('marked-delete');
                    thumbnail.classList.add('selected');
                } else {
                    thumbnail.classList.remove('marked-delete');
                }
            }
            
            // Sort the array for consistent display
            selectedForDeletion.sort((a, b) => a - b);
            
            // Update the display of selected items
            updateSelectedDisplay();
        }
        
        // Function to update the display of selected items
        function updateSelectedDisplay() {
            const selectedContainer = document.getElementById('SELECTED_ID_PLACEHOLDER');
            const codeBlock = selectedContainer.querySelector('.selected-code');
            
            if (selectedForDeletion.length > 0) {
                selectedContainer.style.display = 'block';
                
                // Create the delete_files function call string
                const deleteCallString = `delete_handler([${selectedForDeletion.join(', ')}])`;
                codeBlock.textContent = deleteCallString;
                
                // Enable buttons
                document.getElementById('clear-selection-btn').disabled = false;
                document.getElementById('download-selected-btn').disabled = false;
            } else {
                selectedContainer.style.display = 'none';
                document.getElementById('clear-selection-btn').disabled = true;
                document.getElementById('download-selected-btn').disabled = true;
            }
        }
        
        // Function to clear selection
        function clearSelection() {
            // Remove marked-delete class from all thumbnails
            selectedForDeletion.forEach(idx => {
                const thumbnail = document.querySelector(`[data-idx="${idx}"] .thumbnail`);
                thumbnail.classList.remove('marked-delete');
                
                // Keep the selection class if it's the currently selected item
                if (idx === currentSelectedIdx) {
                    thumbnail.classList.add('selected');
                }
            });
            
            // Clear the array
            selectedForDeletion = [];
            
            // Update the display
            updateSelectedDisplay();
        }
        
        // Function to download a file
        function downloadFile(filePath, event) {
            if (event) event.stopPropagation();
            
            // Create a temporary link to download the file
            const link = document.createElement('a');
            link.href = filePath;
            link.download = filePath.split('/').pop();
            document.body.appendChild(link);
            link.click();
            document.body.removeChild(link);
        }
        
        // Function to download selected files
        function downloadSelected() {
            if (selectedForDeletion.length === 0) return;
            
            // Download each file sequentially with a slight delay to avoid browser issues
            selectedForDeletion.forEach((idx, index) => {
                if (idx >= 0 && idx < allFiles.length) {
                    setTimeout(() => {
                        const filePath = allFiles[idx].path;
                        downloadFile(filePath, null);
                    }, index * 300);
                }
            });
            
            showStatus(`Downloading ${selectedForDeletion.length} file(s)...`, 'success');
        }
        
        // Function to show status messages
        function showStatus(message, type) {
            const statusEl = document.getElementById('status-message');
            statusEl.textContent = message;
            statusEl.className = 'status-msg status-' + type;
            statusEl.style.display = 'block';
            
            setTimeout(() => {
                statusEl.style.display = 'none';
            }, 3000);
        }
        
        // Return public methods
        return {
            previewFile,
            toggleDeleteMark,
            clearSelection,
            downloadFile,
            downloadSelected
        };
    })();
    </script>
    """
    
    # Replace placeholders with actual values
    js_code = js_code.replace('ALLFILES_JSON_PLACEHOLDER', files_json)
    js_code = js_code.replace('PREVIEW_ID_PLACEHOLDER', preview_id)
    js_code = js_code.replace('SELECTED_ID_PLACEHOLDER', selected_id)
    
    # Add the JavaScript to the HTML
    html_code += js_code
    
    # Display the gallery
    display(HTML(html_code))
    
    # Return the delete handler function to be used by JavaScript
    return delete_handler

# Display the gallery and make the delete_handler available
delete_handler = create_media_gallery()